In [33]:
import json

In [34]:
import requests

In [35]:
import geopandas as gpd
import pandas as pd

## Open Geojson

In [36]:
croatia_gdf = gpd.read_file("data/hr.json")

In [37]:
county_name_map = {
    "Medimurska": "County of Međimurje",
    "Zagrebacka": "County of Zagreb",
    "Krapinsko-Zagorska": "County of Krapina-Zagorje",
    "Sisacko-Moslavacka": "County of Sisak-Moslavina",
    "Karlovacka": "County of Karlovac",
    "Varaždinska": "County of Varaždin",
    "Koprivničko-Križevačka": "County of Koprivnica-Križevci",
    "Bjelovarsko-bilogorska": "County of Bjelovar-Bilogora",
    "Primorsko-Goranska": "County of Primorje-Gorski kotar",
    "Licko-Senjska": "County of Lika-Senj",
    "Viroviticko-Podravska": "County of Virovitica-Podravina",
    "Požeško-slavonska županija": "County of Požega-Slavonia",
    "Brodsko-Posavska": "County of Slavonski Brod-Posavina",
    "Zadarska": "County of Zadar",
    "Osjecko-Baranjska": "County of Osijek-Baranja",
    "Šibensko-Kninska": "County of Šibenik-Knin",
    "Vukovarsko-Srijemska": "County of Vukovar-Srijem",
    "Splitsko-Dalmatinska": "County of Split-Dalmatia",
    "Istarska": "County of Istria",
    "Dubrovacko-Neretvanska": "County of Dubrovnik-Neretva",
    "Grad Zagreb": "City of Zagreb",
}

In [38]:
croatia_gdf["name"] = croatia_gdf["name"].map(county_name_map)

## Get Open Population data

In [39]:
url = "https://web.dzs.hr/PxWeb/api/v1/en/Stanovništvo/Procjene stanovništva/SP22_2.px"

In [40]:
data = {
    "query": [
        {"code": "Starost", "selection": {"filter": "item", "values": ["0"]}},
        {"code": "Godina", "selection": {"filter": "item", "values": ["12", "22"]}},
        {"code": "Spol", "selection": {"filter": "item", "values": ["0"]}},
    ],
    "response": {"format": "JSON-stat"},
}

In [41]:
r = requests.post(url, data=json.dumps(data))

In [42]:
json_data = r.json()
indexes = json_data["dataset"]["dimension"]["Prostorna jedinica"]["category"]["index"]
labels = json_data["dataset"]["dimension"]["Prostorna jedinica"]["category"]["label"]
data = json_data["dataset"]["value"]

In [43]:
list_len = int(len(data) / 2)

In [44]:
data_list = [
    {"name": labels[key], "2013": data[:list_len][indx], "2023": data[list_len:][indx]}
    for key, indx in indexes.items()
]

In [45]:
population_df = pd.DataFrame(data_list)
population_df["DIFF"] = population_df["2023"] - population_df["2013"]
population_df["PERCENT"] = (
    (population_df["DIFF"] / population_df["2013"]) * 100
).round(decimals=2)

## Merge data and export

In [47]:
croatia_pop_gdf = croatia_gdf.merge(population_df, on="name", how="left")

In [49]:
croatia_pop_gdf = croatia_pop_gdf.to_crs(3765)
croatia_pop_gdf.to_file("data/croatian_population.gpkg")